In [1]:
%cd ../..

/Users/eli/AnacondaProjects/electropysiology


In [2]:
import electropysiology as epys
import scipy.io as sio

In [3]:
data = sio.loadmat('data.mat', simplify_cells=True, struct_as_record=False)

/Users/eli/miniforge3/envs/ephys/lib/python3.11/site-packages/scipy/io/matlab/_mio.py:227: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [8]:
dt = data['lfpSchema']['smpInterval'][0]

In [12]:
times = data['lfpSchema']['index'][0]